# 目录

1. 损失函数
2. Label Smoothing
3. Focal Loss
4. 优化器
5. 对抗训练
6. FGM
7. PGD

### 1.损失函数

**损失函数**是用来评价模型的**预测值**和**真实值**的不一致程度，损失函数越小，说明模型的鲁棒性越好。

#### 二分类交叉熵
sigmoid公式：
$$\sigma(z)=\frac{1}{1+e^{-z}}$$

#### 二分类交叉熵
$$CrossEntropy(p,q)=-(p(x_1)logq(x_1) + p(x_2)logq(x_2))$$
==> 可以改写为：\
$$CrossEntropy(p,q)=-(plogq + (1-p)log(1-p))$$
其中，
$$p(x_1) + p(x_2) = 1, q(x_1) + q(x_2) = 1$$

#### 多分类交叉熵

其中$c_i$为某个类别，有n个label，$1<=c_i<=n$，设训练数据中类别的概率分布为$p(c_{i})$, 那么目标分布$p(c_{i})$和预测分布$q(c_{i})$的交叉熵为：

$$H(p, q) = - \sum _{i}p(c_i)logq(c_i)$$

每个训练样本所属的类别是已知的，并且每个样本只会属于一个类别（概率为1），属于其他类别的概率为0.\
具体的，可以假设有三分类任务，三分类:Cat,Pig,Dog.假设有一个训练样本的类别为猫。则有：\
P(Cat)=1\
P(Pig)=0\
P(Dog)=0\
通过预测得到的三个分类的概率分别为：\
q(Cat)=0.6\
q(Pig)=0.2\
q(Dog)=0.2\
计算p和q的交叉熵
H(p,q) = -(P(Cat)logq(Cat) + P(Pig)logq(Pig) + P(Dog)logq(Dog))
       = -(1*log0.6 + 0* log0.2 + 0*log0.2)
       = - (1*log0.6) = - logq(Cat)  -- 负对数似然
  
在计算交叉熵时，只需要计算和训练样本对应类别预测概率的值，其它项都是0
这种编码就叫做one-hot编码
Cat = [1, 0, 0]
Pig = [0, 1, 0]
Dog = [0, 0, 1]

##### 参考：
[一文搞懂交叉熵损失](https://www.cnblogs.com/wangguchangqing/p/12068084.html)

## 2.Label Smoothing

![labelsmoothing1](https://img-blog.csdnimg.cn/20210716084858735.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

前提：使用one-hot编码
使用cross-entropy有两个问题：
1. 无法保证模型的泛化能力，容易过拟合。
2. 直觉上，模型对于自己的预测过度自信。

labelsmoothing是分类问题中缓解标注错误的一种方法。

![Labelsmoothing1](https://img-blog.csdnimg.cn/20210715163612615.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

具体的，可以假设有三分类任务，三分类:Cat,Pig,Dog.假设有一个训练样本的类别为猫。则有：\
P(Cat)=1\
P(Pig)=0\
P(Dog)=0\
假设smoothing parameter = 0.1
Cat = (1 - 0.1)[1, 0, 0] + 0.1/3
Cat = [0.9, 0, 0] + 0.1/3
    = [0.933.., 0.033..., 0.033...] 
    [1, 0, 0]

## 3.Focal loss

![focal loss1](https://img-blog.csdnimg.cn/2021071223144792.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

![focal loss](https://img-blog.csdnimg.cn/20210716090444283.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

## focal loss 定义
![focal loss1](https://img-blog.csdnimg.cn/20210715194842146.png)

在交叉熵的基础上添加了一个可调节因子$(1-p_{t})^{\gamma}$, 且$\gamma > 0$
1. 当一个样本被误分类并且$p_{t}$很小时，调节因子因子接近1，$FL = -log(p_{t})$, 并且loss不受影响。
2. 当$p_{t} -〉1$ ，调节因子变为0，FL=0， 降低分类良好样本的权重。
焦点参数${\gamma}$平滑的调整了简单样本被降权的概率。当${\gamma}$=0是，FL=CE。
简单样本：可以理解为正确分类的样本

# alpha-focal loss
![alpha2](https://img-blog.csdnimg.cn/20210715194939601.png)

![梯度下降](https://img-blog.csdnimg.cn/20210716095718709.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

## 4.优化器

![梯度下降变体1](https://img-blog.csdnimg.cn/20210712221612491.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

考虑到 参数更新的准确性 和 时间，从而作出权衡

![batch](https://img-blog.csdnimg.cn/20210716101606165.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

缺点：Batch gradient descent 需要一整数据集的梯度来进行一次更新，因此速度比较慢，不允许模型在线更新。

优点：Batch gradient descent 保证收敛到凸误差表面的全局最小值，非凸表面的局部最小值。

![stochastic](https://img-blog.csdnimg.cn/20210716101730376.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

优点：速度快，可以用来在线更新参数

缺点：更新时，方差高，导致目标函数大幅波动

![Mini-batch](https://img-blog.csdnimg.cn/2021071610185151.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

优点：
速度比batch gradient descent 快；每次利用n个样本，减少了参数更新的方差，更稳定。

当n=1，mini-batch = SGD

![鞍点震荡](https://img-blog.csdnimg.cn/20210712225349599.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

1. 选择合适的学习率比较难。学习率太小会导致收敛速度慢，学习率太大会导致损失函数在最小值附近波动甚至发散。
2. learning rate需要定义，对所有参数都执行相同的更新。
3. 在鞍点处震荡

<img src="https://img-blog.csdnimg.cn/20210620161957409.jpg?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70#pic_center" width="50%">

![Momentum](https://img-blog.csdnimg.cn/20210716103049708.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

SGD在遇到ravines（曲面的一个方向比另一个方向更陡）问题，会被困住。SGD就是发生震荡。这种情况下，就可以使用Momentum解决。


在SGD中加入$\gamma v_{t-1}$ 可以使得在梯度方向不变的维度上速度变快，梯度方向改变的维度上 更新速度变慢，这样就可以加快收敛并减少震荡。

# Adagrad
![Adagrad](https://img-blog.csdnimg.cn/2021071610474648.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

Adagrad 针对 -- learning rate需要定义，对所有参数都执行相同的更新。挑战提出的算法。\
Adagrad是学习率适应参数，对不频繁出现的参数执行比较大的更新，对频繁出现的参数执行较小的更新。因此，Adagrad非常适合处理稀疏数据。\
Adagrad中，每个时间步长t中，对每个参数$\theta_{i}$使用不同的学习率

# Adam
![adam](https://img-blog.csdnimg.cn/20210712230429223.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

Adam是计算每个参数的自适应学习率的方法。

AdamW = Adam+L2正则化

## Lookahead
![lookahead1](https://img-blog.csdnimg.cn/20210714173407795.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

![lookahead2](https://img-blog.csdnimg.cn/20210714174419425.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

Lookahead维护一组慢权重和一组快权重。快权重通过应用优化算法A从example里进行更新。在使用A更新k步后，通过权重空间（快权重的起点和终点）线性插值，将插值更新给慢权重，每次慢权重更新后，快权重都会重置为当前的慢权重。

![Lookahead3](https://img-blog.csdnimg.cn/20210714181925605.png)

## Lamb
![LAMB1](https://img-blog.csdnimg.cn/20210714223942407.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

![LAMB2](https://img-blog.csdnimg.cn/20210714225813983.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

LAMB:模型在进行大批量数据训练时，能够维持梯度更新的精度。

### WarmUp

模型刚开始训练的时候，先使用一个较小的学习率，训练一些epochs，等模型稳定时再修改为预先设置的学习率。

为什么使用Warmup?
模型随机初始化，若选择一个较大的学习率，可能会带来模型的不稳定，选择Warmup先训练几个epochs, 之后，模型趋于稳定，等模型稳定之后在选择预先设置的学习率可以加快模型的收敛速度，模型效果最佳。

## 5.对抗训练

![对抗样本](https://img-blog.csdnimg.cn/20210713144115288.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

对抗样本：对人类看起来一样，对模型来说预测结果却完全不一样的样本。

1.相对于原始输入（词向量），所添加的扰动是微小的
2.模型犯错

对抗训练：在训练集中增加一些对抗样本，希望增强模型对于对抗样本的鲁棒性

![对抗训练](https://img-blog.csdnimg.cn/20210716113626200.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

## 对抗训练方法
### Fast Gradient Method(FGM)
对于每个x:
1. 计算x的前向loss, 反向传播得到梯度；
2. 根据embeddign矩阵计算的梯度计算出r, 并加到当前embedding上，相当于x+r
3. 计算x+r的前向loss, 反向传播得到梯度，然后累加到(1)的梯度上；
4. 将embedding恢复为（1）时的embedding；
5. 根据（3）的梯度对参数进行更新。

### Projected Gradient Descent(PGD)
FGM是一下子算出了对抗扰动，这样得到的扰动不一定是最优的。因此PGD进行了改进，多迭代了K/t次，慢慢找到最优的扰动。对于每个x:
1. 计算x的前向loss, 反向传播得到梯度；
  对于每步t：
  2. 根据embeddign矩阵计算的梯度计算出r, 并加到当前embedding上，相当于x+r；
  3. t如果不是最后一步，将梯度归0， 根据2的x+r计算前后向并得到梯度
  4. t是最后一步，恢复1的梯度，计算最后的x+r并将梯度累加到(1)上
5. 将embedding恢复为（1）时的embedding；
6. 根据（4）的梯度对参数进行更新。